In [1]:
def sequence_extractor(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    from Bio import SeqIO
    from Bio.Seq import Seq
    import os
    
    out_Seq = ''
    
    
    #Genome folder
    entries = os.listdir("/mnt/f/Genomes_2023-12-26/"+Species)
    
    #Get genome file from Genome folder
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    
    #Read the genome file
    fasta_file = open(("/mnt/f/Genomes_2023-12-26/"+Species+"/"+Genome_name),'r')
    
    #Extract the sequence 
    for record in SeqIO.parse(fasta_file,"fasta"):
        if record.id == Scaff:
            sequence = str(record.seq)
            out_Seq = Seq(sequence[start-1:end])
            if reverse_c == 1:
                out_Seq = out_Seq.reverse_complement()
                
    if len(out_Seq) < 10000: #fixing error due to mistake in typing the coordinate, change this for longer sequence
#        print (out_Seq)
        if frame == 1 :
            out_trans = out_Seq[0:]
        if frame == 2 :
            out_trans = out_Seq[1:]
        if frame == 3 :
            out_trans = out_Seq[2:]
    else:
        print ("too long")
        assert(False)
        
        
    #    break
    fasta_file.close()
    if trans == 1:
        return (out_trans.translate())
    else:
        return(out_Seq)
#

In [2]:
import os
from Bio import SeqIO

In [3]:
#%reset -f
blast_location = "/mnt/h/My Drive/Circadian Rhythm Genes Project/8.For Phylogeny/6.Heliconiinae_Danainae_Nymphalinae"
# query_species = "Pieris_brassicae"


species_list = os.listdir(f"{blast_location}/1.Blast_result")
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")


query_file = SeqIO.parse(f"{blast_location}/1.Query/query.txt", 'fasta')
query_name_list = []

for records in query_file:
    query_name_list.append(records.id)



for species in species_list:
    # print(species_name)
    # species = species_name.split(".")[1]
    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    
#Run for each query in the query list
    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
        Length_switch = "0"
        with open(f"{blast_location}/1.Blast_result/{species}/{species}_blast_out.txt",'r') as tblast_out:
            lines_in_file = tblast_out.readlines()
        #print(lines_in_file)

        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0

        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0
        query_length = 0

        for lines in lines_in_file:

#             print(lines)
            if query_name in lines:
            #Initialize that results can now be checked
                result_section_switch = 1
                query_species_split = lines.split(" ")[1].split("_")
#                 print(query_species_split)
                query_species = str(query_species_split[0]+"_"+query_species_split[1].rstrip())
                print(query_species)
            if result_section_switch == 1 and "Lambda" in lines:
            #This block indicates end of the results block in blast output
                result_section_switch == 0
                
                break

            if result_section_switch == 1:
            #While checking the result
                if "Length=" in lines and Length_switch == "0":
                #Get query length from the blast output
                    
                    query_length = int(lines.split("=")[1].rstrip())
                    
                    Length_switch = 1 #Indicated length has been acquired
                    
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
        
                if ">" in lines:
                #Start of the first result
                    scaff = lines.split(" ")[0][1:] #Scaffold from the result
#                     if scaff_old != "Intial_value" and scaff_old != scaff:
#                         error = "Y"
                    scaff_old = scaff
                    
                if "Query" in lines and "=" not in lines:
                #Read the query line in output
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                        #Query start coordinate fixed
            
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting query stop coordinates for multiline result
    #                print (stop_coor)
                    
                if "Sbjct" in lines:
                #Read the blast target line
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                    #Keep getting target stop coordinates for multiline result
                
                

        
        if break_switch == 1:
            break

        if start_coor < stop_coor:
            complement = "0" #Forward complement
            
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1" #Reverse complement
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            error = "Y"
        
        
        
        

        #Check if the length of target (blast hit) is significantly smaller than query
        if length < query_length - 0.2*query_length:
            error = "Y"

        old_trans = ''
#             print(start, stop)


# #Check for Met or STOP at the start and end
#         while True and (start != 0 or stop != 0):
#             if (query_name == query_name_list[0] or query_name == query_name_list[-1]):
#                 frame = 1
#                 translated_sequence = (sequence_extractor(species, scaff, int(complement), start, stop,frame ))

#                 if translated_sequence == old_trans:
#                     break
                    
# #Check for Met at the beginning of first exon

#                 if query_name == query_name_list[0] :
#                     if translated_sequence[0] != "M":
#                         print(translated_sequence)
# #                             print(start, stop)
#                         if complement == "0":
#                             start = int(start) - 3
#                         if complement == "1":
#                             stop = int(stop) + 3
#                         old_trans = translated_sequence
#                     if "*" in translated_sequence:
#                         error = "Y"
#                         break
#                     if translated_sequence[0] == "M":
#                         break

# #Check for stop at the end of last exon

#                 if query_name == query_name_list[-1]:
#                     if translated_sequence[-1] != "*":
#                         print(translated_sequence)
# #                             print(start, stop)
#                         if complement == "0":
#                             stop = int(stop) + 3
#                         if complement == "1":
#                             start = int(start) - 3
#                         old_trans = translated_sequence
#                     if translated_sequence[-1] == "*":

#                         break
#             else:
#                 break
        
        
#Add offset at the beginning and end of each exon
#         if (start != 0 or stop != 0):
            
#             start_modifier = int(query_name.split("Frame")[1][1])
#             stop_modifier = int(query_name.split("rightoh")[1][1])
#         else:
        start_modifier = 0
        stop_modifier = 0            

        seq_length = query_length
        
        
#Get exon start and end by adding or removing codons
        
    #if not the beginning
        if query_start_coor != "1" and query_name != query_name_list[0]:
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)                
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
    
    #For the end
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                

                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
 
        
        
        
#Adding or removing 3' and 5' overhangs for forward and reverse complement
    #For forward complement
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
#             if old_end != 0 and old_end > stop:

#                 error = "Y"
            old_end = stop

    #For reverse complement
        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
#             if old_end != 0 and old_end < stop:
#                 error = "Y"
            old_end = stop

    #Simple check for lenghth
        if start == 0 or stop == 0:
            error = "Y"
            
           
        output_format = str(species.split("\n")[0])+"," + str(scaff) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
        print(output_format)
#         assert False
        Output_Sequence = Output_Sequence + output_format

    print(Output_Sequence)
    output_file = open(f"{blast_location}/1.Blast_result/{species}/{species}_coordinates.csv",'w')
    output_file.write(Output_Sequence)
    output_file.close()


Danaus_plexippus
Hestina_assimilis,CM041741.1,10830529,10831188,0,N,Danaus_plexippus_rps5,1,218,218

Danaus_plexippus
Hestina_assimilis,CM041742.1,4471048,4471572,0,N,Danaus_plexippus_wg,1,175,175

Danaus_plexippus
Hestina_assimilis,CM041743.1,12578636,12579154,1,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Hestina_assimilis,CM041747.1,4301251,4302240,1,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Hestina_assimilis,CM041744.1,9975101,9976324,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Hestina_assimilis,CM041742.1,10168817,10169521,0,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Hestina_assimilis,CM041754.1,6184693,6185688,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Hestina_assimilis,CM041740.1,14848294,14849679,1,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Hestina_assimilis,CM041751.1,14757652,14758428,1,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Hestina_assimilis,CM041759.1,24066591,24068051,1,N,Danaus_plexippus_cat,1,487,487

Danau

Danaus_plexippus
Dircenna_loreta,OY743096.1,11551765,11552283,1,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Dircenna_loreta,OY743098.1,5976319,5977308,1,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Dircenna_loreta,OY743098.1,14336920,14338143,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Dircenna_loreta,OY743094.1,6849069,6849773,1,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Dircenna_loreta,OY743100.1,10358886,10359881,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Dircenna_loreta,OY743087.1,13363659,13365044,1,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Dircenna_loreta,OY743090.1,10081776,10082552,0,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Dircenna_loreta,OY743092.1,10699515,10700975,0,N,Danaus_plexippus_cat,1,487,487

Danaus_plexippus
Dircenna_loreta,OY743098.1,13473233,13476373,0,N,Danaus_plexippus_cad,1,1047,1047

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Dircenna_loreta,OY743089.1,5222

Danaus_plexippus
Melinaea_menophilus,OU911054.1,15338204,15338863,1,N,Danaus_plexippus_rps5,1,218,218

Danaus_plexippus
Melinaea_menophilus,OU911051.1,15967220,15967744,1,N,Danaus_plexippus_wg,1,175,175

Danaus_plexippus
Melinaea_menophilus,OU911055.1,3984546,3985064,1,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Melinaea_menophilus,OU911039.1,31663118,31664107,1,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Melinaea_menophilus,OU911042.1,21931070,21932293,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Melinaea_menophilus,OU911042.1,1967190,1967894,0,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Melinaea_menophilus,OU911044.1,14179813,14180808,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Melinaea_menophilus,OU911046.1,4415936,4417321,0,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Melinaea_menophilus,OU911042.1,26366594,26367370,1,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Melinaea_menophilus,OU911050.1,8914621,8916081,1,N,Danaus_plexippus_

Danaus_plexippus
Boloria_euphrosyne,OX637874.1,10004312,10005535,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Boloria_euphrosyne,OX637873.1,9499102,9499806,1,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Boloria_euphrosyne,OX637885.1,9096721,9097716,1,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Boloria_euphrosyne,OX637872.1,12755191,12756576,1,N,Danaus_plexippus_ef1,1,451,462

Danaus_plexippus
Boloria_euphrosyne,OX637879.1,734393,735169,0,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Boloria_euphrosyne,OX637870.1,11695751,11697211,1,N,Danaus_plexippus_cat,1,487,487

Danaus_plexippus
Boloria_euphrosyne,OX637874.1,8295273,8298413,1,N,Danaus_plexippus_cad,1,1047,1047

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Boloria_euphrosyne,OX637871.1,9689347,9690003,0,N,Danaus_plexippus_rps5,1,218,218
Boloria_euphrosyne,OX637873.1,3960385,3960909,0,N,Danaus_plexippus_wg,1,175,175
Boloria_euphrosyne,OX637875.1,3867152,3867670,0,N,D

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000006.1,4975991,4976647,0,N,Danaus_plexippus_rps5,1,218,218

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000003.1,9915266,9915790,1,N,Danaus_plexippus_wg,1,175,175

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000045.1,90215,90733,1,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000007.1,3840025,3841014,1,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000027.1,3713096,3714319,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000003.1,5528542,5529246,0,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000054.1,2373749,2374744,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000014.1,6178193,6179578,1,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000040.1,583661,584437,0,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Dryadula_phaetusa,JAOYRS010000001.1,91199

Danaus_plexippus
Heliconius_charithonia,CM060809.1,26136514,26137170,1,N,Danaus_plexippus_rps5,1,218,218

Danaus_plexippus
Heliconius_charithonia,CM060809.1,3427476,3428000,0,N,Danaus_plexippus_wg,1,175,175

Danaus_plexippus
Heliconius_charithonia,CM060824.1,7677855,7678373,0,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Heliconius_charithonia,CM060826.1,10686688,10687677,1,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Heliconius_charithonia,CM060827.1,9415836,9417059,0,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Heliconius_charithonia,CM060809.1,7938320,7939024,1,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Heliconius_charithonia,CM060813.1,3189172,3190167,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Heliconius_charithonia,JALPMF010000083.1,35050,36435,1,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Heliconius_charithonia,CM060825.1,19110396,19111172,1,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Heliconius_charithonia,CM060829.1,6717008,67

Danaus_plexippus
Limenitis_camilla,LR990233.1,9339702,9340358,0,N,Danaus_plexippus_rps5,1,218,218

Danaus_plexippus
Limenitis_camilla,LR990227.1,12195304,12195828,1,N,Danaus_plexippus_wg,1,175,175

Danaus_plexippus
Limenitis_camilla,LR990225.1,4540481,4540999,0,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Limenitis_camilla,LR990234.1,11356646,11357635,0,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Limenitis_camilla,LR990229.1,9800814,9802037,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Limenitis_camilla,LR990227.1,6454175,6454879,0,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Limenitis_camilla,LR990241.1,3878184,3879179,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Limenitis_camilla,LR990226.1,3265884,3267269,0,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Limenitis_camilla,LR990228.1,766375,767151,0,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Limenitis_camilla,LR990224.1,11763180,11764640,0,N,Danaus_plexippus_cat,1,487,487

Danaus_plexip

Danaus_plexippus
Mellicta_athalia,HG992182.1,8214553,8215776,0,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Mellicta_athalia,HG992179.1,14353189,14353893,1,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Mellicta_athalia,HG992193.1,5775216,5776211,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Mellicta_athalia,HG992178.1,5207130,5208515,0,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Mellicta_athalia,HG992188.1,1040679,1041455,0,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Mellicta_athalia,HG992176.1,15721156,15722616,1,N,Danaus_plexippus_cat,1,487,487

Danaus_plexippus
Mellicta_athalia,HG992182.1,10833326,10836463,0,N,Danaus_plexippus_cad,1,1046,1047

Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Mellicta_athalia,HG992177.1,9340769,9341422,1,N,Danaus_plexippus_rps5,1,218,218
Mellicta_athalia,HG992179.1,5869745,5870272,0,N,Danaus_plexippus_wg,1,175,175
Mellicta_athalia,HG992181.1,5969215,5969733,0,N,Danaus_plexippus_rp

Danaus_plexippus
Vanessa_atalanta,NC_061872.1,8779456,8780112,0,N,Danaus_plexippus_rps5,1,218,218

Danaus_plexippus
Vanessa_atalanta,NC_061873.1,10726250,10726774,1,N,Danaus_plexippus_wg,1,175,175

Danaus_plexippus
Vanessa_atalanta,NC_061874.1,10532448,10532966,1,N,Danaus_plexippus_rps2,1,173,173

Danaus_plexippus
Vanessa_atalanta,NC_061879.1,8441146,8442135,1,N,Danaus_plexippus_mdh,1,330,330

Danaus_plexippus
Vanessa_atalanta,NC_061876.1,8573686,8574909,1,N,Danaus_plexippus_idh,1,408,408

Danaus_plexippus
Vanessa_atalanta,NC_061873.1,5744540,5745244,0,N,Danaus_plexippus_ddc,1,235,235

Danaus_plexippus
Vanessa_atalanta,NC_061886.1,3530930,3531925,0,N,Danaus_plexippus_gapdh,1,332,332

Danaus_plexippus
Vanessa_atalanta,NC_061871.1,3005310,3006695,0,N,Danaus_plexippus_ef1,1,462,462

Danaus_plexippus
Vanessa_atalanta,NC_061880.1,586261,587037,0,N,Danaus_plexippus_hcl,1,259,259

Danaus_plexippus
Vanessa_atalanta,NC_061902.1,6353214,6354674,1,N,Danaus_plexippus_cat,1,487,487

Danaus_plexippu